In [258]:
from abc import abstractmethod
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from pandas import DataFrame
from datetime import datetime, timedelta
import time
import zipfile, io

In [263]:
class Get:
    def __init__(self):
        self.headers = {"User-Agent": "Mozilla/5.0"}

    def read(self, **params) -> str:
        try:
            url = self.url()
        except:
            url = self.url
        print(url)
        resp = requests.get(url, headers=self.headers, params=params)
        return resp.text

In [264]:
class WebAPI(Get):
    def read(self, **params) -> str:
        params.update(delimiter=self.delimiter, prefix=self.prefix)
        return super().read(**params)
    
    @property
    def delimiter(self):
        return '/'

    @property
    @abstractmethod
    def prefix(self):
        return NotImplementedError

In [265]:
class USDT_kline(WebAPI):
    url_str = 'https://s3-ap-northeast-1.amazonaws.com/data.binance.vision?delimiter=/&prefix=data/futures/um/daily/klines/'
    prefix_str = 'data/futures/um/daily/klines/'
    
    def load(self, symbol: str):
        self.url_str = USDT_kline.url_str + symbol + '/1d/'
        self.prefix_str = USDT_kline.prefix_str + symbol + '/1d/'
        result = self.read()
        soup = bs(result, "xml")
        elements = soup.select('Key')
        klines = [element.text for element in elements if element.text.find('.CHECKSUM') is -1]
        files = [kline.replace(self.prefix_str, '').replace('.zip', '.csv') for kline in klines]
        
        columns = [chr(code) for code in range(65,77)]
        columns.insert(0, 'USDT_name')
        
        result = pd.DataFrame(columns=columns)
        i = 0
        for kline, file in zip(klines, files):
            time.sleep(0.5)
            url = "https://data.binance.vision/" + kline
            resp = requests.get(url, verify=False, stream=True)
            z = zipfile.ZipFile(io.BytesIO(resp.content))
            z.extractall("./USDT/")
            df = pd.read_csv('./USDT/'+file)
            row = [*df.keys()]
            row.insert(0, file.replace('.csv', ''))
            result.loc[i] = row
            i+=1
            if i == 5: break
        return result

    def url(self):
        return self.url_str

    def prefix(self):
        return self.prefix_str

In [269]:
class USDT_list(WebAPI):
    def load(self):
        result = self.read()
        soup = bs(result, "xml")
        elements = soup.select('Prefix')
        symbols = [element.text for element in elements]
        return symbols

    @property
    def url(self):
        return "https://s3-ap-northeast-1.amazonaws.com/data.binance.vision?delimiter=/&prefix=data/futures/um/daily/klines/"

    def prefix(self):
        return 'data/futures/um/daily/klines/'

In [271]:
class PublicData:
    def __init__(self):
        self.USDT_kline = USDT_kline()
        self.USDT_list = USDT_list()

In [273]:
print(PublicData().USDT_list.load())
print(PublicData().USDT_kline.load('1000BTTCUSDT'))

https://s3-ap-northeast-1.amazonaws.com/data.binance.vision?delimiter=/&prefix=data/futures/um/daily/klines/
['data/futures/um/daily/klines/', 'data/futures/um/daily/klines/1000BTTCUSDT/', 'data/futures/um/daily/klines/1000FLOKIUSDT/', 'data/futures/um/daily/klines/1000LUNCBUSD/', 'data/futures/um/daily/klines/1000LUNCUSDT/', 'data/futures/um/daily/klines/1000PEPEUSDT/', 'data/futures/um/daily/klines/1000SHIBBUSD/', 'data/futures/um/daily/klines/1000SHIBUSDT/', 'data/futures/um/daily/klines/1000XECUSDT/', 'data/futures/um/daily/klines/1INCHUSDT/', 'data/futures/um/daily/klines/AAVEUSDT/', 'data/futures/um/daily/klines/ACHUSDT/', 'data/futures/um/daily/klines/ADABUSD/', 'data/futures/um/daily/klines/ADAUSDT/', 'data/futures/um/daily/klines/AGIXBUSD/', 'data/futures/um/daily/klines/AGIXUSDT/', 'data/futures/um/daily/klines/AGLDUSDT/', 'data/futures/um/daily/klines/AKROUSDT/', 'data/futures/um/daily/klines/ALGOUSDT/', 'data/futures/um/daily/klines/ALICEUSDT/', 'data/futures/um/daily/kline

c:\Users\jihyuk\.conda\envs\fba\lib\site-packages\urllib3\connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.binance.vision'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\Users\jihyuk\.conda\envs\fba\lib\site-packages\urllib3\connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.binance.vision'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\Users\jihyuk\.conda\envs\fba\lib\site-packages\urllib3\connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.binance.vision'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning

                    USDT_name              A          B          C          D  \
0  1000BTTCUSDT-1d-2022-01-26  1643155200000  0.0020890  0.0021000  0.0019630   
1  1000BTTCUSDT-1d-2022-01-27  1643241600000  0.0020200  0.0024290  0.0018640   
2  1000BTTCUSDT-1d-2022-01-28  1643328000000  0.0020520  0.0020960  0.0019680   
3  1000BTTCUSDT-1d-2022-01-29  1643414400000  0.0020730  0.0021850  0.0020540   
4  1000BTTCUSDT-1d-2022-01-30  1643500800000  0.0021490  0.0021640  0.0020670   

           E           F              G                 H       I           J  \
0  0.0020200  3008436212  1643241599999   6137792.1776000   57521  1219794163   
1  0.0020520  9755239817  1643327999999  20440816.9447480  115200  4347668349   
2  0.0020730  3193788995  1643414399999   6503584.6852130   54764  1517782762   
3  0.0021500  3397045959  1643500799999   7173612.4017120   56097  1554349277   
4  0.0020900  2199008466  1643587199999   4646814.8665810   30062  1011233647   

                 K  L  
0 

c:\Users\jihyuk\.conda\envs\fba\lib\site-packages\urllib3\connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.binance.vision'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
